<a href="https://colab.research.google.com/github/AtharvBhat/PIAYN/blob/main/PIAYN_TextClassificationLRABaseline_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

As usual, we first install HuggingFace Transformers, and Datasets.

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 9.1 MB/s 
     |████████████████████████████████| 895 kB 61.7 MB/s 
     |████████████████████████████████| 6.5 MB 56.6 MB/s 
     |████████████████████████████████| 77 kB 8.2 MB/s 


In [2]:
!pip install -q datasets

     |████████████████████████████████| 325 kB 8.3 MB/s 
     |████████████████████████████████| 212 kB 71.2 MB/s 
     |████████████████████████████████| 1.1 MB 69.3 MB/s 
     |████████████████████████████████| 136 kB 16.3 MB/s 
     |████████████████████████████████| 127 kB 55.9 MB/s 
     |████████████████████████████████| 94 kB 1.4 MB/s 
     |████████████████████████████████| 271 kB 52.9 MB/s 
     |████████████████████████████████| 144 kB 63.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!git clone https://github.com/google-research/long-range-arena.git
!git clone https://github.com/mlpen/Nystromformer.git

Cloning into 'long-range-arena'...
remote: Enumerating objects: 474, done.
remote: Counting objects: 100% (474/474), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 474 (delta 330), reused 423 (delta 282), pack-reused 0
Receiving objects: 100% (474/474), 153.28 KiB | 10.95 MiB/s, done.
Resolving deltas: 100% (330/330), done.
Cloning into 'Nystromformer'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 292 (delta 140), reused 147 (delta 42), pack-reused 0
Receiving objects: 100% (292/292), 10.82 MiB | 7.57 MiB/s, done.
Resolving deltas: 100% (140/140), done.


## Set up output folders

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

#Set the task
task = 'text'

baseDir = "/content/drive/MyDrive"
piaynDir = os.path.join(baseDir, "PIAYN")

if not os.path.exists(piaynDir):
  os.makedirs(piaynDir)

piaynTaskDir = os.path.join(piaynDir, task)

if not os.path.exists(piaynTaskDir):
  os.makedirs(piaynTaskDir)

piaynTaskModelDir = os.path.join(piaynTaskDir, "models")

if not os.path.exists(piaynTaskModelDir):
  os.makedirs(piaynTaskModelDir)

piaynTaskDataDir = os.path.join(piaynTaskDir, "data")

if not os.path.exists(piaynTaskDataDir):
  os.makedirs(piaynTaskDataDir)    

## Prepare data

Here we take a small portion of the IMDB dataset, a binary text classification dataset ("is a movie review positive or negative?").

In [6]:
from datasets import load_dataset
from Nystromformer.LRA.datasets import text
import pickle, numpy as np

#Create Train test and dev MAP files for the imdb dataset
!python Nystromformer/LRA/datasets/text.py
!cp text.train.pickle /content/drive/MyDrive/PIAYN/text/data
!cp text.dev.pickle /content/drive/MyDrive/PIAYN/text/data
!cp text.test.pickle /content/drive/MyDrive/PIAYN/text/data

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQUFKFI/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
INFO:tensorflow:Finished preprocessing


INFO:tensorflow:Finished preprocessing


INFO:tensorflow:<MapDataset element_spec={'Source': TensorSpec(shape=(), dtype=tf.string, name=None), 'Target': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


INFO:tensorflow:<MapDataset element_spec={'Source': TensorSpec(shape=(), dtype=tf.string, name=None), 'Target': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


2022-04-11 09:18:40.403467: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


In [7]:
from Nystromformer.LRA.code import lra_config
from Nystromformer.LRA.code.dataset import LRADataset
#from Nystromformer.LRA.code.run_tasks import training_config
from torch.utils.data import DataLoader, RandomSampler

#get training config
training_config = lra_config.config[task]["training"]

#Check Train Config
print('Training Config: ', training_config)

#get pre-defined model config
model_config = lra_config.config[task]['model']

#Check model Config
print('Model Config: ', model_config)

#Get the dataset
train_dataset = LRADataset(piaynTaskDataDir + f"/{task}.train.pickle", False)
val_dataset = LRADataset(piaynTaskDataDir + f"/{task}.dev.pickle", False)
test_dataset = LRADataset(piaynTaskDataDir + f"/{task}.test.pickle", False)

#Create DataLoader iterators
ds_iter = {
    "train":enumerate(DataLoader(train_dataset, 
                                 #Sample batches randomly for number of specified steps
                                 sampler = RandomSampler(train_dataset, 
                                                         replacement=True, 
                                                         num_samples= training_config["num_train_steps"]*lra_config.config[task]['dataset']['train']), 
                                 batch_size = training_config["batch_size"], 
                                 drop_last = True)),
    "dev":enumerate(DataLoader(val_dataset, batch_size = 1, drop_last = True)),
    "test":enumerate(DataLoader(test_dataset, batch_size = 1, drop_last = True)),
}


Training Config:  {'batch_size': 32, 'learning_rate': 0.0001, 'warmup': 8000, 'lr_decay': 'linear', 'weight_decay': 0, 'eval_frequency': 500, 'num_train_steps': 20000, 'num_eval_steps': 781}
Model Config:  {'learn_pos_emb': True, 'tied_weights': False, 'embedding_dim': 64, 'transformer_dim': 64, 'transformer_hidden_dim': 128, 'head_dim': 32, 'num_head': 2, 'num_layers': 2, 'vocab_size': 512, 'max_seq_len': 4000, 'dropout_prob': 0.1, 'attention_dropout': 0.1, 'pooling_mode': 'MEAN', 'num_classes': 2}
Loaded /content/drive/MyDrive/PIAYN/text/data/text.train.pickle... size=25000
Loaded /content/drive/MyDrive/PIAYN/text/data/text.dev.pickle... size=25000
Loaded /content/drive/MyDrive/PIAYN/text/data/text.test.pickle... size=25000


In [8]:
#Check sizes of batches
batch = next((ds_iter['train']))
for k,v in batch[1].items():
  print(k,v.shape)

input_ids_0 torch.Size([32, 1024])
mask_0 torch.Size([32, 1024])
label torch.Size([32])


In [9]:
!pip install ml_collections

     |████████████████████████████████| 77 kB 5.1 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=11650901b483c8324d0d240c84e47c15b163639d4a08195a38ebc2f243b9b7a8
  Stored in directory: /root/.cache/pip/wheels/b7/da/64/33c926a1b10ff19791081b705879561b715a8341a856a3bbd2
Successfully built ml-collections


In [10]:
import ml_collections
def get_config():
  """Get the default hyperparameter configuration."""
  config = ml_collections.ConfigDict()
  config.batch_size = 32
  config.eval_frequency = 100
  config.num_train_steps = 20000
  config.num_eval_steps = -1
  config.learning_rate = 0.05
  config.weight_decay = 1e-1
  config.max_target_length = 200
  config.max_eval_target_length = 200
  config.sampling_temperature = 0.6
  config.sampling_top_k = 20
  config.max_predict_token_length = 50
  config.save_checkpoints = True
  config.restore_checkpoints = True
  config.checkpoint_freq = 10000
  config.random_seed = 0
  config.prompt = ""
  config.factors = "constant * linear_warmup * rsqrt_decay"
  config.warmup = 8000
  config.classifier_pool = "CLS"

  config.max_length = 1000

  config.emb_dim = 256
  config.num_heads = 4
  config.num_layers = 4
  config.qkv_dim = 256
  config.mlp_dim = 1024

  config.trial = 0  # dummy for repeated runs.
  return config

## Define model

Next, we define our model, and put it on the GPU.

In [11]:
from transformers import PerceiverForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
from transformers import PerceiverConfig
#get default perceiver config
configuration = PerceiverConfig()

#Update the Perceiver configurations with Preset model configs
configuration.update(model_config)

#Print Updated Perceiver Configuration
print(configuration)

PerceiverConfig {
  "attention_dropout": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "audio_samples_per_frame": 1920,
  "cross_attention_shape_for_attention": "kv",
  "cross_attention_widening_factor": 1,
  "d_latents": 1280,
  "d_model": 768,
  "dropout_prob": 0.1,
  "embedding_dim": 64,
  "head_dim": 32,
  "hidden_act": "gelu",
  "image_size": 56,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "learn_pos_emb": true,
  "max_position_embeddings": 2048,
  "max_seq_len": 4000,
  "model_type": "perceiver",
  "num_blocks": 1,
  "num_classes": 2,
  "num_cross_attention_heads": 8,
  "num_frames": 16,
  "num_head": 2,
  "num_latents": 256,
  "num_layers": 2,
  "num_self_attends_per_block": 26,
  "num_self_attention_heads": 8,
  "output_shape": [
    1,
    16,
    224,
    224
  ],
  "pooling_mode": "MEAN",
  "qk_channels": null,
  "samples_per_patch": 16,
  "self_attention_widening_factor": 1,
  "tied_weights": false,
  "train_size": [
    368,
    496
  ],
  "transformer_dim": 

In [13]:
def initialize_model(config):
  #Initialize Model
  model = PerceiverForSequenceClassification(config)
  
  #Get Model Parameter Counts
  pytorch_total_params = sum(p.numel() for p in model.parameters())
  pytorch_total_params_Trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print('Total Parameters: ', pytorch_total_params, '\nTrainable Parameters: ', pytorch_total_params_Trainable)  

  return model#.to(device)                                          

In [16]:
#Change the perceiver configurations to get total parameters within 10% of BERT
configuration.num_labels = 2
#configuration.id2label = id2label
#configuration.label2id = label2id
configuration.num_self_attends_per_block = 10
#configuration.num_latents = 256


model = initialize_model(configuration).to(device)

Total Parameters:  117010690 
Trainable Parameters:  117010690


## Train the model

Here we train the model using native PyTorch.

In [17]:
device

device(type='cuda')

In [18]:
from transformers import AdamW
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from datasets import load_metric

best_score = 0 
prev_score = 0
maxPatience = 3
currentPatience = 0

#steps = int(training_config["num_train_steps"]/20000)
steps = training_config["num_train_steps"]

optimizer = AdamW(model.parameters(), 
                  lr = training_config["learning_rate"],
                  betas = (0.9, 0.999), 
                  eps = 1e-6, 
                  weight_decay = training_config["weight_decay"])

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer = optimizer,
    max_lr = training_config["learning_rate"],
    pct_start = training_config["warmup"] / training_config["num_train_steps"],
    anneal_strategy = training_config["lr_decay"],
    total_steps = training_config["num_train_steps"]
)

#amp_scaler = torch.cuda.amp.GradScaler() if model_config["mixed_precision"] else None

model.to(device)
model.train()
for step in tqdm(range(steps)):  # Perform gradient updates for multiple steps
    
    model.train()
    
    #print("Step:", step)
    #for batch in tqdm(train_dataloader):
    batch = next(ds_iter['train'])[1]

    # get the inputs; 
    inputs = batch["input_ids_0"].to(device)
    attention_mask = batch["mask_0"].to(device)
    labels = batch["label"].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs=inputs, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    # evaluate
    predictions = outputs.logits.argmax(-1).cpu().detach().numpy()
    accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
    print(f"Loss: {loss.item()}, Accuracy: {accuracy}")

    #delete intermediate variables to free up GPU space
    del loss, outputs, inputs, attention_mask, labels, predictions, accuracy


    #Every 1000 steps validate and save model
    if (step+1)%1000 == 0:
      
      model.eval()

      val_accuracy = load_metric("accuracy")

      #reset dev iterator
      ds_iter['dev'] = enumerate(DataLoader(val_dataset, batch_size = 1, drop_last = True))

      with torch.no_grad():
        for i, batch in tqdm(ds_iter['dev']):
              
          # get the inputs; 
          inputs = batch["input_ids_0"].to(device)
          attention_mask = batch["mask_0"].to(device)
          labels = batch["label"].to(device)

          # forward pass
          outputs = model(inputs=inputs, attention_mask=attention_mask)
          logits = outputs.logits 
          predictions = logits.argmax(-1).cpu().detach().numpy()
          references = batch["label"].numpy()
          val_accuracy.add_batch(predictions=predictions, references=references)

          #delete intermediate variables to free up GPU space
          del logits, outputs, inputs, attention_mask, labels, predictions, references

      final_score = val_accuracy.compute()['accuracy']
      print("Validation Accuracy:", final_score)

      if final_score > best_score:
        best_score = final_score
        torch.save(model.to('cpu').state_dict(), piaynTaskModelDir + '/trainPerceiverClassifier.pkl')
        model.to(device)
      else:
        pass  

      if final_score < prev_score:
        currentPatience += 1
        if currentPatience > maxPatience:
          print('Patience Limit reached! Stopping early!')
          torch.save(model.to('cpu').state_dict(), piaynTaskModelDir + '/trainPerceiverClassifierStep_' + str(step + 1) + '.pkl')
          break  
      else:
        currentPatience = 0

#Save the trained model
#torch.save(model.to('cpu').state_dict(), piaynTaskModelDir + '/trainPerceiverClassifier.pkl')
#print('Model Saved!')
#Free up space from gpu

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/20000 [00:00<?, ?it/s]

Loss: 0.6687682271003723, Accuracy: 0.625
Loss: 0.9814210534095764, Accuracy: 0.5625
Loss: 0.8406741619110107, Accuracy: 0.46875
Loss: 0.788811981678009, Accuracy: 0.3125
Loss: 0.6797896027565002, Accuracy: 0.59375
Loss: 0.6812975406646729, Accuracy: 0.625
Loss: 0.7615514993667603, Accuracy: 0.53125
Loss: 0.8072317242622375, Accuracy: 0.40625
Loss: 0.6853353381156921, Accuracy: 0.5625
Loss: 0.6971949338912964, Accuracy: 0.53125
Loss: 0.7496728897094727, Accuracy: 0.4375
Loss: 0.7712297439575195, Accuracy: 0.46875
Loss: 0.7579114437103271, Accuracy: 0.46875
Loss: 0.7032926678657532, Accuracy: 0.53125
Loss: 0.7117739915847778, Accuracy: 0.5
Loss: 0.6819590926170349, Accuracy: 0.65625
Loss: 0.6926707029342651, Accuracy: 0.59375
Loss: 0.7142492532730103, Accuracy: 0.46875
Loss: 0.7697224020957947, Accuracy: 0.3125
Loss: 0.6502132415771484, Accuracy: 0.65625
Loss: 0.7336119413375854, Accuracy: 0.375
Loss: 0.6770724654197693, Accuracy: 0.5625
Loss: 0.698885440826416, Accuracy: 0.5
Loss: 0.69

0it [00:00, ?it/s]

Validation Accuracy: 0.60592
Loss: 0.6964958906173706, Accuracy: 0.5
Loss: 0.6168798804283142, Accuracy: 0.71875
Loss: 0.6157164573669434, Accuracy: 0.71875
Loss: 0.6656029224395752, Accuracy: 0.59375
Loss: 0.6586484313011169, Accuracy: 0.71875
Loss: 0.6923871636390686, Accuracy: 0.53125
Loss: 0.7075892090797424, Accuracy: 0.4375
Loss: 0.6874770522117615, Accuracy: 0.5
Loss: 0.6550852060317993, Accuracy: 0.59375
Loss: 0.6449632048606873, Accuracy: 0.65625
Loss: 0.6568574905395508, Accuracy: 0.59375
Loss: 0.6445696949958801, Accuracy: 0.65625
Loss: 0.6546179056167603, Accuracy: 0.65625
Loss: 0.7686256766319275, Accuracy: 0.40625
Loss: 0.655394434928894, Accuracy: 0.53125
Loss: 0.5511068105697632, Accuracy: 0.78125
Loss: 0.7031997442245483, Accuracy: 0.46875
Loss: 0.6417163014411926, Accuracy: 0.59375
Loss: 0.666761040687561, Accuracy: 0.625
Loss: 0.6386962532997131, Accuracy: 0.53125
Loss: 0.6542953252792358, Accuracy: 0.65625
Loss: 0.6878058314323425, Accuracy: 0.5625
Loss: 0.656300365

...

Validation Accuracy: 0.58
Loss: 0.002988385269418359, Accuracy: 1.0
Loss: 0.0011403399985283613, Accuracy: 1.0
Loss: 0.0025606572162359953, Accuracy: 1.0
Loss: 0.0020871595479547977, Accuracy: 1.0
Loss: 0.007034730166196823, Accuracy: 1.0
Loss: 0.00853560771793127, Accuracy: 1.0
Loss: 0.001214811229147017, Accuracy: 1.0
Loss: 0.0012659889180213213, Accuracy: 1.0
Loss: 0.0051554786041378975, Accuracy: 1.0
Loss: 0.0368540994822979, Accuracy: 0.96875
Loss: 0.00119737908244133, Accuracy: 1.0
Loss: 0.002655802993103862, Accuracy: 1.0
Loss: 0.0024334366898983717, Accuracy: 1.0
Loss: 0.0019582114182412624, Accuracy: 1.0
Loss: 0.005272907670587301, Accuracy: 1.0
Loss: 0.027383917942643166, Accuracy: 0.96875
Loss: 0.018603699281811714, Accuracy: 1.0
Loss: 0.00240676780231297, Accuracy: 1.0
Loss: 0.004159399773925543, Accuracy: 1.0
Loss: 0.0037784501910209656, Accuracy: 1.0
Loss: 0.003137541702017188, Accuracy: 1.0
Loss: 0.019324561581015587, Accuracy: 1.0
Loss: 0.0009869510540738702, Accuracy: 

0it [00:00, ?it/s]

Validation Accuracy: 0.58232
Loss: 0.016810160130262375, Accuracy: 1.0
Loss: 0.000510681769810617, Accuracy: 1.0
Loss: 0.00043214455945417285, Accuracy: 1.0
Loss: 0.0009080482996068895, Accuracy: 1.0
Loss: 0.000420214346377179, Accuracy: 1.0
Loss: 0.02735094353556633, Accuracy: 0.96875
Loss: 0.00029965382418595254, Accuracy: 1.0
Loss: 0.0007723775343038142, Accuracy: 1.0
Loss: 0.2558284401893616, Accuracy: 0.96875
Loss: 0.014769063331186771, Accuracy: 1.0
Loss: 0.0005270195542834699, Accuracy: 1.0
Loss: 0.000736175337806344, Accuracy: 1.0
Loss: 0.0003602727083489299, Accuracy: 1.0
Loss: 0.0007470579585060477, Accuracy: 1.0
Loss: 0.0007690945640206337, Accuracy: 1.0
Loss: 0.0005335355526767671, Accuracy: 1.0
Loss: 0.2446347177028656, Accuracy: 0.96875
Loss: 0.0004236029926687479, Accuracy: 1.0
Loss: 0.0005563633749261498, Accuracy: 1.0
Loss: 0.0007751874509267509, Accuracy: 1.0
Loss: 0.0006122811464592814, Accuracy: 1.0
Loss: 0.02527031861245632, Accuracy: 0.96875
Loss: 0.00059912854339

0it [00:00, ?it/s]

Validation Accuracy: 0.58264


In [19]:
!nvidia-smi

Mon Apr 11 16:17:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    51W / 250W |  11797MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
del model
torch.cuda.empty_cache()
!nvidia-smi

Mon Apr 11 16:17:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    50W / 250W |   3493MiB / 16280MiB |     88%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Evaluate the model

Finally, we evaluate the model on the test set. We use the Datasets library to compute the accuracy.

In [22]:
from tqdm.notebook import tqdm
from datasets import load_metric

accuracy = load_metric("accuracy")

#load best performing model checkpoint
model = PerceiverForSequenceClassification(configuration)
model.load_state_dict(torch.load(piaynTaskModelDir + '/trainPerceiverClassifier.pkl'))
model.to(device)

model.eval()

with torch.no_grad():
  for i, batch in tqdm(ds_iter['test']):
        
        # get the inputs; 
        inputs = batch["input_ids_0"].to(device)
        attention_mask = batch["mask_0"].to(device)
        labels = batch["label"].to(device)

        # forward pass
        outputs = model(inputs=inputs, attention_mask=attention_mask)
        logits = outputs.logits 
        predictions = logits.argmax(-1).cpu().detach().numpy()
        references = batch["label"].numpy()
        accuracy.add_batch(predictions=predictions, references=references)

        #delete intermediate variables to free up GPU space
        del logits, outputs, inputs, attention_mask, labels, predictions, references

final_score = accuracy.compute()
print("Accuracy on test set:", final_score['accuracy'])

0it [00:00, ?it/s]

Accuracy on test set: 0.62964
